# Introduction to Daily Price Data and Alpha Factors

In [13]:
import awswrangler as wr
import boto3
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
pd.set_option('display.max_columns',1000)

## Introduction


There are multiple daily frequency datasets provided by AlgoSeek. Here we will use the Standard Daily Adjusted OHLC (Open High Low Close).

In [46]:
daily = wr.s3.read_csv("s3://algoseekraw/us-equity-standard-daily-ohlc-adjusted/")

KeyboardInterrupt: 

In [3]:
daily.head()

,TradeDate,SecId,Ticker,Open,High,Low,Close,MarketHoursVolume,CumulativePriceFactor,CumulativeVolumeFactor,AdjustmentFactor,AdjustmentReason
0,20160104,2234,SLF,30.44,30.58,30.00,30.57,454226,1.498942,1.000000,NaN,NaN
1,20160104,18665,TNP,7.89,7.85,7.53,7.66,398554,3.226711,2.000000,NaN,NaN
2,20160104,18679,ACE,115.02,115.41,112.52,113.73,2818882,1.230675,1.000000,NaN,NaN
3,20160104,19690,LOGI,15.02,15.09,14.61,14.83,355490,1.187021,1.000000,NaN,NaN
4,20160104,20946,DB,23.20,23.48,23.02,23.49,2554117,1.473137,1.158348,NaN,NaN


In [4]:
ratio = wr.s3.read_parquet('s3://securityfundamentalsdata/finratio_processed.parquet')

In [14]:
ratio.head()


,gvkey,permno,adate,qdate,public_date,CAPEI,BE,bm,evm,pe_op_basic,pe_op_dil,pe_exi,pe_inc,ps,pcf,dpr,npm,opmbd,opmad,gpm,ptpm,cfm,roa,roe,roce,efftax,aftret_eq,aftret_invcapx,aftret_equity,pretret_noa,pretret_earnat,GProf,equity_invcap,debt_invcap,totdebt_invcap,capital_ratio,int_debt,int_totdebt,cash_lt,invt_act,rect_act,debt_at,debt_ebitda,short_debt,curr_debt,lt_debt,profit_lct,ocf_lct,cash_debt,fcf_ocf,lt_ppent,dltt_be,debt_assets,debt_capital,de_ratio,intcov,intcov_ratio,cash_ratio,quick_ratio,curr_ratio,cash_conversion,inv_turn,at_turn,rect_turn,pay_turn,sale_invcap,sale_equity,sale_nwc,rd_sale,adv_sale,staff_sale,accrual,ret_crsp,gsector,gicdesc,sp500,mktcap,price,ptb,PEG_trailing,divyield,FFI5_desc,FFI5,FFI10_desc,FFI10,FFI12_desc,FFI12,FFI17_desc,FFI17,FFI30_desc,FFI30,FFI38_desc,FFI38,FFI48_desc,FFI48,FFI49_desc,FFI49,dt,TICKER,cusip
0,001000,25881.0,1969-12-31,1970-09-30,1971-01-31,10.949959,10.211,0.427328,1.761483,NaN,NaN,12.676056,13.432836,0.649427,-3.254120,0.0,0.059602,0.132756,0.105129,0.326433,0.101225,0.076701,0.286655,0.287123,0.340907,0.515192,0.210622,0.592462,0.210622,0.483161,0.249587,0.425118,0.876062,0.123938,0.742448,0.123938,0.368051,0.101741,0.069023,0.512418,0.383678,0.304542,1.761483,0.833069,0.755635,0.077455,0.355079,-0.52525,-0.396897,NaN,2.190245,0.141471,0.644365,0.556492,1.812047,4.508604,7.516252,0.091345,0.603863,1.238484,70.633935,5.219896,2.159265,10.278175,13.858005,3.095835,3.533807,11.215357,0.0,0.0,0.0,-0.529999,-0.100000,<NA>,<NA>,0.0,23.895000,9.000,2.340123,NaN,NaN,MANUF,2.0,MANUF,3.0,MANUF,3.0,DURBL,5.0,<NA>,NaN,RUBBR,16.0,RUBBR,15.0,RUBBR,15.0,3925.0,<NA>,<NA>
1,001000,25881.0,1970-12-31,1970-12-31,1971-02-28,35.407426,10.544,0.431071,7.009840,NaN,NaN,19.866071,65.441176,0.651525,24.410640,0.0,0.041425,0.118804,0.088982,0.326591,0.079850,0.071247,0.173289,0.180968,0.188536,0.481215,0.180977,0.234284,0.180968,0.359970,0.144102,0.442631,0.919990,0.080010,1.160021,0.080010,0.723404,0.077136,0.072470,0.477683,0.417451,0.397459,2.468437,0.931027,0.855453,0.040033,0.274866,0.06175,0.052825,-1.286777,2.580667,0.086969,0.684783,0.647982,2.172420,3.209412,4.745882,0.084715,0.569125,1.089615,115.765768,3.201615,1.458608,5.828619,6.257931,3.955589,4.299602,25.817198,0.0,0.0,0.0,-0.021492,0.236111,<NA>,<NA>,0.0,29.536875,11.125,2.801297,NaN,NaN,MANUF,2.0,MANUF,3.0,MANUF,3.0,DURBL,5.0,<NA>,NaN,RUBBR,16.0,RUBBR,15.0,RUBBR,15.0,4017.0,AE,00003210
2,001000,25881.0,1970-12-31,1970-12-31,1971-03-31,31.031228,10.544,0.431071,7.009840,NaN,NaN,17.410714,57.352941,0.570999,21.393595,0.0,0.041425,0.118804,0.088982,0.326591,0.079850,0.071247,0.173289,0.180968,0.188536,0.481215,0.180977,0.234284,0.180968,0.359970,0.144102,0.442631,0.919990,0.080010,1.160021,0.080010,0.723404,0.077136,0.072470,0.477683,0.417451,0.397459,2.468437,0.931027,0.855453,0.040033,0.274866,0.06175,0.052825,-1.286777,2.580667,0.086969,0.684783,0.647982,2.172420,3.209412,4.745882,0.084715,0.569125,1.089615,115.765768,3.201615,1.458608,5.828619,6.257931,3.955589,4.299602,25.817198,0.0,0.0,0.0,-0.021492,-0.123596,<NA>,<NA>,0.0,25.886250,9.750,2.455069,NaN,NaN,MANUF,2.0,MANUF,3.0,MANUF,3.0,DURBL,5.0,<NA>,NaN,RUBBR,16.0,RUBBR,15.0,RUBBR,15.0,4017.0,AE,00003210
3,001000,25881.0,1970-12-31,1970-12-31,1971-04-30,34.213918,10.544,0.431071,7.009840,NaN,NaN,19.196429,63.235294,0.629563,23.587810,0.0,0.041425,0.118804,0.088982,0.326591,0.079850,0.071247,0.173289,0.180968,0.188536,0.481215,0.180977,0.234284,0.180968,0.359970,0.144102,0.442631,0.919990,0.080010,1.160021,0.080010,0.723404,0.077136,0.072470,0.477683,0.417451,0.397459,2.468437,0.931027,0.855453,0.040033,0.274866,0.06175,0.052825,-1.286777,2.580667,0.086969,0.684783,0.647982,2.172420,3.209412,4.745882,0.084715,0.569125,1.089615,115.765768,3.201615,1.458608,5.828619,6.257931,3.955589,4.299602,25.817198,0.0,0.0,0.0,-0.021492,0.102564,<NA>,<NA>,0.0,28.541250,10.750,2.706871,NaN,NaN,MANUF,2.0,MANUF,3.0,MANUF,3.0,DURBL,5.0,<NA>,NaN,RUBBR,16.0,RUBBR,15.0,RUBBR,15.0,4017.0,AE,00003210
4

In [ ]:
ratio = ratio.drop(columns=['Unnamed:_0'])

In [15]:
ratio['gsector'].describe()

count     1098197
unique         11
top            45
freq       208592
Name: gsector, dtype: object

In [29]:
ratio['gsector']

0          <NA>
1          <NA>
2          <NA>
3          <NA>
4          <NA>
           ... 
2653807      35
2653808      35
2653809      35
2653810      45
2653811      45
Name: gsector, Length: 2653812, dtype: string

In [16]:
ratio['gsector'].isna().sum()

1555615

In [17]:
ratio.isna().sum()

gvkey               0
permno              0
adate           66539
qdate          136215
public_date         0
                ...  
FFI49_desc      76669
FFI49           76669
dt                  0
TICKER          52639
cusip           20811
Length: 100, dtype: int64

In [18]:
r = ratio.dropna()

In [19]:
r.head()

,gvkey,permno,adate,qdate,public_date,CAPEI,BE,bm,evm,pe_op_basic,pe_op_dil,pe_exi,pe_inc,ps,pcf,dpr,npm,opmbd,opmad,gpm,ptpm,cfm,roa,roe,roce,efftax,aftret_eq,aftret_invcapx,aftret_equity,pretret_noa,pretret_earnat,GProf,equity_invcap,debt_invcap,totdebt_invcap,capital_ratio,int_debt,int_totdebt,cash_lt,invt_act,rect_act,debt_at,debt_ebitda,short_debt,curr_debt,lt_debt,profit_lct,ocf_lct,cash_debt,fcf_ocf,lt_ppent,dltt_be,debt_assets,debt_capital,de_ratio,intcov,intcov_ratio,cash_ratio,quick_ratio,curr_ratio,cash_conversion,inv_turn,at_turn,rect_turn,pay_turn,sale_invcap,sale_equity,sale_nwc,rd_sale,adv_sale,staff_sale,accrual,ret_crsp,gsector,gicdesc,sp500,mktcap,price,ptb,PEG_trailing,divyield,FFI5_desc,FFI5,FFI10_desc,FFI10,FFI12_desc,FFI12,FFI17_desc,FFI17,FFI30_desc,FFI30,FFI38_desc,FFI38,FFI48_desc,FFI48,FFI49_desc,FFI49,dt,TICKER,cusip
517,001004,54594.0,1999-05-31,1999-05-31,1999-07-31,22.787483,370.905,0.685877,7.646158,13.948675,14.135906,14.135906,14.135906,0.629625,20.263565,0.224977,0.045391,0.102876,0.084290,0.207314,0.065124,0.063978,0.135191,0.117614,0.156961,0.302997,0.132946,0.116920,0.132946,0.184072,0.133014,0.261924,0.643100,0.356900,0.357728,0.356900,0.103597,0.103407,0.020594,0.599188,0.323311,0.249589,1.920281,0.002316,0.433320,0.451676,0.544076,0.164328,0.071207,-0.266643,3.851431,0.487831,0.551305,0.488249,1.228687,3.244358,4.167663,0.047527,1.173407,2.927575,146.702697,2.562716,1.314119,5.603572,6.335738,1.810815,2.815759,2.743682,0.0,0.0,0.0,-0.018818,-0.067879,20,Industrials,0.0,578.018187,21.0625,1.558400,0.454208,0.016142,CNSMR,1.0,SHOPS,7.0,SHOPS,9.0,MACHN,11.0,WHLSL,26.0,WHLSL,33.0,WHLSL,41.0,WHLSL,42.0,14395.0,AIR,00036110
518,001004,54594.0,1999-05-31,1999-05-31,1999-08-31,23.090184,370.905,0.685877,7.646158,14.155629,14.345638,14.345638,14.345638,0.637988,20.532739,0.224977,0.045391,0.102876,0.084290,0.207314,0.065124,0.063978,0.135191,0.117614,0.156961,0.302997,0.132946,0.116920,0.132946,0.184072,0.133014,0.261924,0.643100,0.356900,0.357728,0.356900,0.103597,0.103407,0.020594,0.599188,0.323311,0.249589,1.920281,0.002316,0.433320,0.451676,0.544076,0.164328,0.071207,-0.266643,3.851431,0.487831,0.551305,0.488249,1.228687,3.244358,4.167663,0.047527,1.173407,2.927575,146.702697,2.562716,1.314119,5.603572,6.335738,1.810815,2.815759,2.743682,0.0,0.0,0.0,-0.018818,0.014837,20,Industrials,0.0,585.696375,21.3750,1.579101,0.460947,0.015906,CNSMR,1.0,SHOPS,7.0,SHOPS,9.0,MACHN,11.0,WHLSL,26.0,WHLSL,33.0,WHLSL,41.0,WHLSL,42.0,14395.0,AIR,00036110
519,001004,54594.0,1999-05-31,1999-05-31,1999-09-30,19.444366,370.905,0.685877,7.646158,11.920530,12.080537,12.080537,12.080537,0.537253,17.290727,0.224977,0.045391,0.102876,0.084290,0.207314,0.065124,0.063978,0.135191,0.117614,0.156961,0.302997,0.132946,0.116920,0.132946,0.184072,0.133014,0.261924,0.643100,0.356900,0.357728,0.356900,0.103597,0.103407,0.020594,0.599188,0.323311,0.249589,1.920281,0.002316,0.433320,0.451676,0.544076,0.164328,0.071207,-0.266643,3.851431,0.487831,0.551305,0.488249,1.228687,3.244358,4.167663,0.047527,1.173407,2.927575,146.702697,2.562716,1.314119,5.603572,6.335738,1.810815,2.815759,2.743682,0.0,0.0,0.0,-0.018818,-0.157895,20,Industrials,0.0,493.218000,18.0000,1.329769,0.388166,0.018889,CNSMR,1.0,SHOPS,7.0,SHOPS,9.0,MACHN,11.0,WHLSL,26.0,WHLSL,33.0,WHLSL,41.0,WHLSL,42.0,14395.0,AIR,00036110
520,001004,54594.0,1999-05-31,1999-08-31,1999-10-31,18.685325,380.332,0.649343,7.670858,10.697115,11.199664,10.836039,10.836039,0.423141,-63.104359,0.218032,0.039680,0.089998,0.073892,0.179346,0.056788,0.055787,0.135077,0.119356,0.160526,0.302997,0.135047,0.116911,0.135047,0.184337,0.130529,0.268133,0.642351,0.357649,0.370078,0.357649,0.111500,0.107755,0.030515,0.587686,0.311652,0.258291,1.919625,0.033586,0.432743,0.452465,0.563606,-0.041992,0.071207,-0.266643,3.967326,0.490701,0.551680,0.490210,1.230549,3.131481,3.969230,0.070514,1.225462,2.972154,126.631204,2.942282,1.495058,6.760830,7.390167,2.142112,3.334799,3.175410,0.0,0.0,0.0,-0.069640,-0.0

In [30]:
r['gsector']

517        20
518        20
519        20
520        20
521        20
           ..
2653357    15
2653358    15
2653359    15
2653360    15
2653361    15
Name: gsector, Length: 78508, dtype: string

In [26]:
daily['Ticker']

0             SLF
1             TNP
2             ACE
3            LOGI
4              DB
            ...  
14008433    MTBpH
14008434     DCPE
14008435     DBND
14008436     MBNE
14008437     INQQ
Name: Ticker, Length: 14008438, dtype: object

In [31]:
iid = wr.s3.read_parquet('s3://taqdata/wrds_iid/wrds_iid_2019.parquet')

In [33]:
iid=iid.drop(columns=['Unnamed:_0'])

In [38]:
iid=iid.dropna()

In [39]:
iid.head()

,DATE,SYM_ROOT,SYM_SUFFIX,BuyNumTrades_LR,SellNumTrades_LR,total_trade,BuyVol_LR,SellVol_LR,total_vol,avg_buy_price_LR,avg_sell_price_LR,buy_dv_LR,sell_dv_LR,total_dv_LR,vwavg_buy_price_LR,vwavg_sell_price_LR,symbol,CTime,CEX,CSize,CPrc,TR_SEQNUM_close,C_official,OTime,OEX,OSize,OPrc,TR_SEQNUM_open,O_official,size_1pm,ttime_1pm,ptime_1pm,size_4pm,ttime_4pm,ptime_4pm,stime_open,ttime_open,ptime_open,stime_close,ttime_close,ptime_close,nbbot_1pm,nbo_1pm,nbb_1pm,nboqty_1pm,nbbqty_1pm,nbbot_4pm,nbo_4pm,nbb_4pm,nboqty_4pm,nbbqty_4pm,nbbot_after_open,nbo_after_open,nbb_after_open,nboqty_after_open,nbbqty_after_open,nbbot_before_close,nbo_before_close,nbb_before_close,nboqty_before_close,nbbqty_before_close,mid_1pm,mid_4pm,mid_after_open,mid_before_close,price_low_m,price_high_m,avg_price_m,total_vol_m,total_dollar_m,total_n_trades_m,vw_price_m,price_low_b,price_high_b,avg_price_b,total_vol_b,total_dollar_b,total_n_trades_b,vw_price_b,price_low_a,price_high_a,avg_price_a,total_vol_a,total_dollar_a,total_n_trades_a,vw_price_a,ret_mkt_m,n_outside_nbbo_trade,n_oddlot_trade,oddlot_vol,oddlot_dollar,n_iso_trade,iso_vol,iso_dollar,BuyNumTrades_tick,BuyVol_tick,buy_dv_tick,avg_buy_price_tick,SellNumTrades_tick,SellVol_tick,sell_dv_tick,avg_sell_price_tick,total_trade_tick,total_vol_tick,total_dv_tick,vwavg_buy_price_tick,vwavg_sell_price_tick,BuyNumTrades_wrds,BuyVol_wrds,buy_dv_wrds,avg_buy_price_wrds,SellNumTrades_wrds,SellVol_wrds,sell_dv_wrds,avg_sell_price_wrds,total_trade_wrds,total_vol_wrds,total_dv_wrds,vwavg_buy_price_wrds,vwavg_sell_price_wrds,QuotedSpread_Dollar_tw,QuotedSpread_Percent_tw,BestOfrDepth_Dollar_tw,BestBidDepth_Dollar_tw,BestOfrDepth_Share_tw,BestBidDepth_Share_tw,EffectiveSpread_Dollar_Ave,EffectiveSpread_Percent_Ave,EffectiveSpread_Dollar_DW,EffectiveSpread_Dollar_SW,EffectiveSpread_Percent_DW,EffectiveSpread_Percent_SW,DollarRealizedSpread_LR_Ave,PercentRealizedSpread_LR_Ave,DollarPriceImpact_LR_Ave,PercentPriceImpact_LR_Ave,DollarRealizedSpread_LR_SW,DollarRealizedSpread_LR_DW,PercentRealizedSpread_LR_SW,PercentRealizedSpread_LR_DW,DollarPriceImpact_LR_SW,DollarPriceImpact_LR_DW,PercentPriceImpact_LR_SW,PercentPriceImpact_LR_DW,ivol_t,ivol_q,bs_ratio_num,bs_ratio_vol,TSignSqrtDVol1,TSignSqrtDVol2,n_obs,n5_pos,n30_pos,HIndex,var_ratio1,var_ratio2,var_ratio3,var_ratio4,var_ratio5,BuyNumTrades_Retail,BuyVol_Retail,buy_dv_Retail,SellNumTrades_Retail,SellVol_Retail,sell_dv_Retail,total_trade_retail,total_vol_retail,total_dv_retail,avg_buy_price_Retail,avg_sell_price_Retail,bs_ratio_retail_num,bs_ratio_retail_vol,BuyNumTrades_Inst20k,BuyVol_Inst20k,buy_dv_Inst20k,SellNumTrades_Inst20k,SellVol_Inst20k,sell_dv_Inst20k,total_trade_Inst20k,total_vol_Inst20k,total_dv_Inst20k,avg_buy_price_Inst20k,avg_sell_price_Inst20k,bs_ratio_Inst20k_num,bs_ratio_Inst20k_vol,BuyNumTrades_Inst50k,BuyVol_Inst50k,buy_dv_Inst50k,SellNumTrades_Inst50k,SellVol_Inst50k,sell_dv_Inst50k,total_trade_Inst50k,total_vol_Inst50k,total_dv_Inst50k,avg_buy_price_Inst50k,avg_sell_price_Inst50k,bs_ratio_Inst50k_num,bs_ratio_Inst50k_vol
998,2019-01-02,BRK,b'B',20330.0,18484.0,38814.0,1724091.0,1901279.0,3625370.0,202.319813,202.281891,3.488353e+08,3.845432e+08,7.333785e+08,202.330002,202.254986,BRKB,57790.082691,b'N',736963.0,202.80,247462401.0,1.0,34238.224913,b'N',130426.0,202.01,1263201.0,1.0,200.0,46795.234179,202.0300,100.0,57599.947419,202.8500,15.0,34238.227067,202.26,2.0,57781.078381,202.8000,46795.742778,202.04,201.97,200.0,100.0,57599.972537,202.80,202.79,100.0,200.0,34238.227048,202.02,202.00,800.0,2200.0,57702.309584,202.80,202.79,100.0,200.0,202.005,202.795,202.010,202.795,201.1500,204.40,202.302240,4360209.0,8.824082e+08,38799.0,202.377509,198.01,202.74,201.166446,22179.0,4.463160e+06,347.0,201.233602,198.5000,203.00,200.677605,420543.0,8.519278e+07,643.0,202.578036,0.003903,1887.0,19658.0,578166.0,1.169700e+08,17414.0,1405893.0,2.843964e+08,19902.0,2931509.0,5.935816e+08,202.296615,19886.0,1871372.0,3.784726e+08,202.235637,39788.0,4802881.0,9.

In [35]:
tst = wr.s3.read_parquet('s3://securityfundamentalsdata/test_intraday_features.parquet')

In [36]:
tst.head()

,date,minute,ret1min,ret2min,ret3min,ret4min,ret5min,ret6min,ret7min,ret8min,ret9min,ret10min
0,2968,188,0.000376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2968,189,-0.000376,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2968,192,-0.001494,-0.002244,-0.001869,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2968,328,0.000000,-0.001494,-0.002244,-0.001869,NaN,NaN,NaN,NaN,NaN,NaN
4,2968,341,-0.000371,0.000376,-0.001118,-0.001868,-0.001493,NaN,NaN,NaN,NaN,NaN


In [40]:
from bokeh.plotting import figure,show
from bokeh.models import ColumnDataSource, CDSView, BooleanFilter,HoverTool,FactorRange
from bokeh.transform import factor_cmap, cumsum
from bokeh.layouts import row, column, gridplot
from bokeh.models.widgets import Tabs,Panel
from bokeh.io import output_notebook

In [42]:
output_notebook()

Loading BokehJS ...

In [45]:
daily

,TradeDate,SecId,Ticker,Open,High,Low,Close,MarketHoursVolume,CumulativePriceFactor,CumulativeVolumeFactor,AdjustmentFactor,AdjustmentReason
0,1970-01-01 00:00:00.020160104,2234,SLF,30.44,30.58,30.00,30.57,454226,1.498942,1.000000,NaN,NaN
1,1970-01-01 00:00:00.020160104,18665,TNP,7.89,7.85,7.53,7.66,398554,3.226711,2.000000,NaN,NaN
2,1970-01-01 00:00:00.020160104,18679,ACE,115.02,115.41,112.52,113.73,2818882,1.230675,1.000000,NaN,NaN
3,1970-01-01 00:00:00.020160104,19690,LOGI,15.02,15.09,14.61,14.83,355490,1.187021,1.000000,NaN,NaN
4,1970-01-01 00:00:00.020160104,20946,DB,23.20,23.48,23.02,23.49,2554117,1.473137,1.158348,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
14008433,1970-01-01 00:00:00.020220406,7386304,MTBpH,25.95,26.97,25.36,25.90,88610,1.000000,1.000000,NaN,NaN
14008434,1970-01-01 00:00:00.020220406,7387953,DCPE,24.96,24.99,24.83,24.86,39170,1.000000,1.000000,NaN,NaN
14008435,1970-01-01 00:00:00.020220406,7387957,DBND,NaN,NaN,NaN,49.60,128,1.000000,1.000000,NaN,NaN
14008436,1970-01-01 00:00:00.020220406,7387996,MBNE,29.99,29.93,29.93,29.96,167266,1.000000,1.000000,NaN,NaN


In [44]:
daily['TradeDate']=pd.to_datetime(daily['TradeDate'])

In [ ]:

TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

p = figure(x_axis_type="datetime", tools=TOOLS, width=1000, title = "MSFT Candlestick")
p.xaxis.major_label_orientation = pi/4
p.grid.grid_line_alpha=0.3

p.segment(df.date, df.high, df.date, df.low, color="black")
p.vbar(df.date[inc], w, df.open[inc], df.close[inc], fill_color="#D5E1DD", line_color="black")
p.vbar(df.date[dec], w, df.open[dec], df.close[dec], fill_color="#F2583E", line_color="black")

show(p)